In [405]:
import pandas as pd
import numpy as np
from numpy import linalg

from bokeh.models import LinearInterpolator, CategoricalColorMapper, HoverTool, ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure
from ipywidgets import interact
from bokeh.palettes import RdYlGn


output_notebook()

Loading BokehJS ...

In [433]:
x0,y0 = 1,1
a,b,c,d = 1,2,3,4
steps = 8
xvals = []
yvals = []

In [434]:
#Define some useful functions here:
def matrixhit(a,b,c,d,x0,y0,steps):
    xvals = [0]
    yvals = [0]
    ind = [0]
    mat0 = np.array([(a,b),(c,d)])
    newmat = mat0
    if steps == 0:
        return [x0],[y0], ind
    elif steps == 1:
        xvals[0], yvals[0] = x0, y0
        xvals += [np.matmul(mat0, np.array([x0,y0]))[0]]
        yvals += [np.matmul(mat0, np.array([x0,y0]))[1]] 
        ind = [1]*2
        return xvals,yvals,ind
    else:
        xvals[0], yvals[0] = x0, y0
        xvals += [np.matmul(mat0, np.array([x0,y0]))[0]]
        yvals += [np.matmul(mat0, np.array([x0,y0]))[1]]
        newmat = np.matmul(mat0,mat0) 
        for i in range(steps-1):
            xvals += [np.matmul(newmat,np.array([x0,y0]))[0]]
            yvals += [np.matmul(newmat,np.array([x0,y0]))[1]]
            newmat = np.matmul(mat0,newmat)
        ind = [steps]*(steps+1)
        return xvals, yvals, ind

In [435]:
#data = pd.DataFrame({'ind':ind, 'xvals':col1, 'yvals':col2}).set_index('ind')
col1,col2, ind = list(),list(), list()
for i in range(steps):
    col1 += matrixhit(a,b,c,d,x0,y0,i)[0]
    col2 += matrixhit(a,b,c,d,x0,y0,i)[1]
    ind += matrixhit(a,b,c,d,x0,y0,i)[2]
data = pd.DataFrame({'xvals': col1, 'yvals': col2, 'ind': ind}).set_index('ind')
data.head()

,xvals,yvals
ind,,
0,1,1
1,1,1
1,3,7
2,1,1
2,3,7


In [438]:
#size_mapper = LinearInterpolator(
#    x=[data.iloc[:,0].min(), data.iloc[:,1].max()],
#    y=[5, 50]
#)

#color_mapper = CategoricalColorMapper(
#    factors = list(data.iloc[0,:],
#    palette=RdYlGn,
#)

PLOT_OPTS = dict(height=400, x_axis_type='linear', x_range=(np.min(col1)-100,np.max(col1)+100), y_range=(np.min(col2)-100,np.max(col2)+100))

source = ColumnDataSource(dict(
    x=data.loc[1].xvals,
    y=data.loc[1].yvals,
))

hover = HoverTool(show_arrow=False,tooltips=[
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
])

p=figure(
    title='Eigenvectors and Eigenvalues',
    title_location='above',
    toolbar_location='right',
    tools=[hover],
    **PLOT_OPTS)

p.circle(
    x='x',y='y',
    size=8,
    #size={'field': 'xvals', 'transform': size_mapper},
    #color={'field': 'x', 'transform': color_mapper},
    alpha=0.6,
    source=source,
)
show(p, notebook_handle=True)

def update(ind):
    new_data = dict(
        x=data.loc[ind].xvals,
        y=data.loc[ind].yvals,
    )
    source.data = new_data
    p.title.text='xvals'
    push_notebook()

In [440]:
interact(update, ind=(1,steps-1))

interactive(children=(IntSlider(value=4, description='ind', max=7, min=1), Output()), _dom_classes=('widget-interact',))

<function __main__.update>